In [ ]:
#昔のコード
"""
df = pd.DataFrame(index=[], columns=datalist)
for data in datalist:
    X1 = pd.read_excel(data, usecols="C", names=[0])
    Y1 = pd.read_excel(data, usecols="D", names=[0])
    Z1 = pd.read_excel(data, usecols="E", names=[0])
    #X2 = pd.read_excel(data, usecols="E", names=[0])
    #Y2 = pd.read_excel(data, usecols="F", names=[0])
    #Z2 = pd.read_excel(data, usecols="G", names=[0])
    # print(time_data)
    # ファイル名(column)とそれに対応するx軸データをdfに格納
    time_data = pd.concat([X1, Y1, Z1], ignore_index=True)
    # time_data = pd.concat([X1, Y1, Z1], ignore_index=True)
    df[data] = time_data

# print(Y1)
print(time_data)
    
print(df)
plt.plot(df)
plt.show()

# DataFrameをnp.arrayに変換
input_data = df.iloc[:,:-1].values

# tslearnで扱えるデータに変換
ts = to_time_series_dataset(df.T) #nm_data

print(type(ts))
print(ts.shape)
print(ts)

km = TimeSeriesKMeans(n_clusters=3, metric="dtw")
labels = km.fit_predict(ts)

print(labels)

# シルエット係数の計算
silhouette_vals = silhouette_samples(Body, km, metric='euclidean')

# シルエット係数のグラフのプロット
cluster_labels = np.unique(km)
n_clusters = cluster_labels.shape[0]
y_ax_lower, y_ax_upper = 0, 0
yticks = []

"""

In [1]:
#今作成しているコード（TimeSeriesKMeans）
import pandas as pd
import numpy as np
import os
from tslearn.clustering import TimeSeriesKMeans
from tslearn.preprocessing import TimeSeriesScalerMeanVariance
import matplotlib.pyplot as plt

# フォルダパスの設定
folder_path = "C:\\Users\\miyazakipc\\Desktop\\2024年9月10日　実験\\てすりとキネクト（てすり）\\データ分析\\分析用抽出データ"

# 新しい保存先フォルダの設定
save_folder_path = "C:\\Users\\miyazakipc\\Desktop\\2024年9月10日　実験\\てすりとキネクト（てすり）\\データ分析\\ラベル付きデータ"
os.makedirs(save_folder_path, exist_ok=True)

# すべてのCSVファイルのパスを取得
file_list = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.csv')]

# 全てのファイルのデータを格納するリスト
all_data = []

# 各ファイルから座標データを読み込み、リストに追加
for file in file_list:
    data = pd.read_csv(file)
    xyz_data = data[['2', '3', '4']].values
    all_data.append(xyz_data)
   
# データを結合し、全体のデータセットを作成
all_data = np.array(all_data)
print(f"Shape of all_data: {all_data.shape}")
"""
# 時系列データを正規化
scaler = TimeSeriesScalerMeanVariance()
all_data_scaled = scaler.fit_transform(all_data)
print(f"Shape of all_data_scaled: {all_data_scaled.shape}")

# TimeSeriesKMeansによるクラスタリング (DTW距離を使用、クラスター数k=3の例)
n_clusters = 3
model = TimeSeriesKMeans(n_clusters=n_clusters, metric="dtw", random_state=0, n_init=10)
labels = model.fit_predict(all_data_scaled)

# クラスタラベルを各ファイルに追加して保存
for i, file in enumerate(file_list):
    data = pd.read_csv(file)
    data['ClusterLabel'] = labels[i]  # クラスタラベルを追加
    # 新しいファイル名を作成して保存
    new_file_name = os.path.join(save_folder_path, os.path.basename(file).replace(".csv", "lavel.csv"))
    data.to_csv(new_file_name, index=False)
    print(f"ファイル: {new_file_name} - クラスタ: {labels[i]} に保存しました。")

print("全てのファイルが正しく保存されました。")
"""

Shape of all_data: (30, 41, 3)


'\n# 時系列データを正規化\nscaler = TimeSeriesScalerMeanVariance()\nall_data_scaled = scaler.fit_transform(all_data)\nprint(f"Shape of all_data_scaled: {all_data_scaled.shape}")\n\n# TimeSeriesKMeansによるクラスタリング (DTW距離を使用、クラスター数k=3の例)\nn_clusters = 3\nmodel = TimeSeriesKMeans(n_clusters=n_clusters, metric="dtw", random_state=0, n_init=10)\nlabels = model.fit_predict(all_data_scaled)\n\n# クラスタラベルを各ファイルに追加して保存\nfor i, file in enumerate(file_list):\n    data = pd.read_csv(file)\n    data[\'ClusterLabel\'] = labels[i]  # クラスタラベルを追加\n    # 新しいファイル名を作成して保存\n    new_file_name = os.path.join(save_folder_path, os.path.basename(file).replace(".csv", "lavel.csv"))\n    data.to_csv(new_file_name, index=False)\n    print(f"ファイル: {new_file_name} - クラスタ: {labels[i]} に保存しました。")\n\nprint("全てのファイルが正しく保存されました。")\n'

In [ ]:
#エルボー法によるクラスタリング数の計算
distortions = [] 

for i in range(1,11): 
    ts_km = TimeSeriesKMeans(n_clusters=i,metric="dtw",random_state=0) 
    ts_km.fit_predict(all_data_scaled)
    distortions.append(ts_km.inertia_) 

plt.plot(range(1,11),distortions,marker="o") 
plt.xticks(range(1,11)) 
plt.xlabel("Number of clusters") 
plt.ylabel("SSE") 

# グラフの保存先ディレクトリ
graph_save_path = r"C:\\Users\\miyazakipc\\Desktop\\2024年9月10日　実験\\てすりとキネクト（てすり）\\データ分析\\TimeSeriesKMeans_Kinect_Elbow.png"

# グラフの保存
plt.savefig(graph_save_path)
# グラフの表示
plt.show()

In [ ]:
#クラスタラベル毎にプロット
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.cm as cm

# ラベル付きデータのフォルダパス
folder_path = 'C:\\Users\\miyazakipc\\Desktop\\2024年9月10日 実験\\てすりとキネクト（てすり）\\データ分析\\ラベル付きデータ'

# グラフ保存先フォルダパス
save_folder_path ="C:\\Users\\miyazakipc\\Desktop\\2024年9月10日　実験\\てすりとキネクト（てすり）\\データ分析\\クラスタリング結果"
os.makedirs(save_folder_path, exist_ok=True)

# すべてのCSVファイルのパスを取得
file_list = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.csv')]

# クラスタラベルごとのファイルを格納する辞書
cluster_files = {}

# 各ファイルを読み込み、クラスタラベルに従って分類
for file in file_list:
    data = pd.read_csv(file)
    if 'ClusterLabel' in data.columns:
        # クラスタラベルを取得
        cluster_label = data['ClusterLabel'].iloc[0]
        
        # ラベルに応じてファイルを分類
        if cluster_label not in cluster_files:
            cluster_files[cluster_label] = []
        cluster_files[cluster_label].append(file)
    else:
        print(f"File {file} does not have the required 'ClusterLabel' column")

# カラーマップの設定
colors = cm.rainbow(np.linspace(0, 1, len(cluster_files)))

# クラスタラベルごとにプロット
for cluster_label, color in zip(cluster_files.keys(), colors):
    plt.figure(figsize=(12, 8))
    
    # 各ファイルに対してプロット
    for file in cluster_files[cluster_label]:
        data = pd.read_csv(file)
        if '0' in data.columns and '2' in data.columns and '3' in data.columns and '4' in data.columns:
            # Time列をdatetime型に変換
            data["0"] = pd.to_datetime(data["0"])
            # 時間分解能（間隔）を計算
            time_intervals = (data["0"] - data["0"].iloc[0]).dt.total_seconds()
            # 各軸の力データをプロット
            plt.plot(time_intervals, data["2"], color="r", alpha=0.8, label=f'Cluster {cluster_label} - Fx' if file == cluster_files[cluster_label][0] else "")
            plt.plot(time_intervals, data["3"], color="g", alpha=0.8, label=f'Cluster {cluster_label} - Fy' if file == cluster_files[cluster_label][0] else "")
            plt.plot(time_intervals, data["4"], color="b", alpha=0.8, label=f'Cluster {cluster_label} - Fz' if file == cluster_files[cluster_label][0] else "")
        else:
            print(f"File {file} does not have the required columns")
    
    # グラフのタイトルやラベルの設定
    plt.title(f'Cluster {cluster_label} - Force Data (Elements: {len(cluster_files[cluster_label])})', fontsize=16)
    plt.xlabel('Time (seconds)', fontsize=14)
    plt.ylabel('Force (N)', fontsize=14)
    plt.legend(loc='upper right', fontsize=15)
    plt.grid(True, which="both", linestyle="--", linewidth=0.5, alpha=0.7)
    
    # グラフを保存
    graph_save_path = os.path.join(save_folder_path, f'Cluster_{cluster_label}.png')
    plt.savefig(graph_save_path)
    plt.close()
    
    print(f'グラフを {graph_save_path} に保存しました。')


In [ ]:
#全体プロット
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.cm as cm

# ラベル付きデータのフォルダパス
folder_path = 'C:\\Users\\miyazakipc\\Desktop\\2024年9月10日 実験\\てすりとキネクト（てすり）\\データ分析\\ラベル付きデータ'

# グラフ保存先フォルダパス
save_folder_path ="C:\\Users\\miyazakipc\\Desktop\\2024年9月10日　実験\\てすりとキネクト（てすり）\\データ分析\\クラスタリング結果"
os.makedirs(save_folder_path, exist_ok=True)

# すべてのCSVファイルのパスを取得
file_list = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.csv')]

# クラスタラベルごとのファイルを格納する辞書
cluster_files = {}

# 各ファイルを読み込み、クラスタラベルに従って分類
for file in file_list:
    data = pd.read_csv(file)
    if 'ClusterLabel' in data.columns:
        # クラスタラベルを取得
        cluster_label = data['ClusterLabel'].iloc[0]
        
        # ラベルに応じてファイルを分類
        if cluster_label not in cluster_files:
            cluster_files[cluster_label] = []
        cluster_files[cluster_label].append(file)
    else:
        print(f"File {file} does not have the required 'ClusterLabel' column")

# カラーマップの設定
cmap = plt.get_cmap('tab10')
colors = [cmap(i) for i in range(len(cluster_files))]

# プロットの準備
plt.figure(figsize=(12, 8))

# クラスタラベルごとにプロット
for cluster_label, color in zip(cluster_files.keys(), colors):
    
    # 各ファイルに対してプロット
    for file in cluster_files[cluster_label]:
        data = pd.read_csv(file)
        if '0' in data.columns and '2' in data.columns and '3' in data.columns and '4' in data.columns:
            # Time列をdatetime型に変換
            data["0"] = pd.to_datetime(data["0"])
            # 時間分解能（間隔）を計算
            time_intervals = (data["0"] - data["0"].iloc[0]).dt.total_seconds()
            # 各軸の力データをプロット
            plt.plot(time_intervals, data["2"], color=color, alpha=0.5, label=f'Cluster {cluster_label} - Fx' if file == cluster_files[cluster_label][0] else "")
            plt.plot(time_intervals, data["3"], color=color, alpha=0.5, label=f'Cluster {cluster_label} - Fy' if file == cluster_files[cluster_label][0] else "")
            plt.plot(time_intervals, data["4"], color=color, alpha=0.5, label=f'Cluster {cluster_label} - Fz' if file == cluster_files[cluster_label][0] else "")
        else:
            print(f"File {file} does not have the required columns")

# グラフのタイトルやラベルの設定
plt.title('Force Data by Cluster', fontsize=16)
plt.xlabel('Time (seconds)', fontsize=14)
plt.ylabel('Force (N)', fontsize=14)
plt.legend(loc='upper right', fontsize=12)
plt.grid(True, which="both", linestyle="--", linewidth=0.5, alpha=0.7)
# グラフを保存
graph_save_path = os.path.join(save_folder_path, "all_cluster.png")
plt.savefig(graph_save_path)
plt.show()


In [ ]:
#軸ごとにプロット
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.cm as cm

# ラベル付きデータのフォルダパス
folder_path = 'C:\\Users\\miyazakipc\\Desktop\\2024年9月10日 実験\\てすりとキネクト（てすり）\\データ分析\\ラベル付きデータ'

# グラフ保存先フォルダパス
save_folder_path ="C:\\Users\\miyazakipc\\Desktop\\2024年9月10日　実験\\てすりとキネクト（てすり）\\データ分析\\クラスタリング結果"
os.makedirs(save_folder_path, exist_ok=True)

# すべてのCSVファイルのパスを取得
file_list = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.csv')]

# クラスタラベルごとのファイルを格納する辞書
cluster_files = {}

# 各ファイルを読み込み、クラスタラベルに従って分類
for file in file_list:
    data = pd.read_csv(file)
    if 'ClusterLabel' in data.columns:
        # クラスタラベルを取得
        cluster_label = data['ClusterLabel'].iloc[0]
        
        # ラベルに応じてファイルを分類
        if cluster_label not in cluster_files:
            cluster_files[cluster_label] = []
        cluster_files[cluster_label].append(file)
    else:
        print(f"File {file} does not have the required 'ClusterLabel' column")

# カラーマップの設定
cmap = plt.get_cmap('tab10')
colors = [cmap(i) for i in range(len(cluster_files))]

# プロットの準備
plt.figure(figsize=(12, 8))

# クラスタラベルごとにプロット
for cluster_label, color in zip(cluster_files.keys(), colors):
    
    # 各ファイルに対してプロット
    for file in cluster_files[cluster_label]:
        data = pd.read_csv(file)
        if '0' in data.columns and '2' in data.columns and '3' in data.columns and '4' in data.columns:
            # Time列をdatetime型に変換
            data["0"] = pd.to_datetime(data["0"])
            # 時間分解能（間隔）を計算
            time_intervals = (data["0"] - data["0"].iloc[0]).dt.total_seconds()
            # 各軸の力データをプロット
            #plt.plot(time_intervals, data["2"], color=color, alpha=0.5, label=f'Cluster {cluster_label} - Fx' if file == cluster_files[cluster_label][0] else "")
            #plt.plot(time_intervals, data["3"], color=color, alpha=0.5, label=f'Cluster {cluster_label} - Fy' if file == cluster_files[cluster_label][0] else "")
            plt.plot(time_intervals, data["4"], color=color, alpha=0.5, label=f'Cluster {cluster_label} - Fz' if file == cluster_files[cluster_label][0] else "")
        else:
            print(f"File {file} does not have the required columns")

# グラフのタイトルやラベルの設定
plt.title('Force Data by Cluster', fontsize=16)
plt.xlabel('Time (seconds)', fontsize=14)
plt.ylabel('Force (N)', fontsize=14)
plt.legend(loc='upper right', fontsize=12)
plt.grid(True, which="both", linestyle="--", linewidth=0.5, alpha=0.7)
# グラフを保存
graph_save_path = os.path.join(save_folder_path, "z_by_cluster.png")
plt.savefig(graph_save_path)
plt.show()


In [ ]:
# クラスタリング評価
from sklearn.metrics import silhouette_samples, silhouette_score
from tslearn.metrics import cdist_dtw


# DTW距離行列の計算
dtw_distances = cdist_dtw(all_data_scaled)

# シルエットスコアの計算（三次元座標をそのまま使用。距離行列)
silhouette_avg = silhouette_score(dtw_distances, labels, metric="precomputed")
print(f'平均シルエットスコア: {silhouette_avg}')

# 各サンプルのシルエットスコアの計算
silhouette_vals = silhouette_samples(dtw_distances, labels, metric="precomputed")

# シルエット係数のグラフのプロット
cluster_labels = np.unique(labels)
n_clusters = cluster_labels.shape[0]
y_ax_lower, y_ax_upper = 0, 0
yticks = []

for i, c in enumerate(cluster_labels):
    c_silhouette_vals = silhouette_vals[labels == c]
    c_silhouette_vals.sort()
    y_ax_upper += len(c_silhouette_vals)
    color = cm.jet(float(i) / n_clusters)
    plt.barh(range(y_ax_lower, y_ax_upper), c_silhouette_vals, height=1.0, edgecolor='none', color=color)
    yticks.append((y_ax_lower + y_ax_upper) / 2)
    y_ax_lower += len(c_silhouette_vals)
    print(f"クラスタ {c}: 要素数 {len(c_silhouette_vals)}, シルエット係数 {c_silhouette_vals}")

plt.axvline(silhouette_avg, color="red", linestyle="--")
plt.yticks(yticks, cluster_labels)
plt.ylabel('Cluster')
plt.xlabel('Silhouette coefficient')
plt.title('Silhouette Coefficients by Cluster (DTW)')
plt.savefig("C:\\Users\\miyazakipc\\Desktop\\2024年9月10日　実験\\てすりとキネクト（てすり）\\データ分析\\Silhouette_coefficient.png")
plt.show()

print('シルエット分析')

In [ ]:
# クラスタリング評価
from sklearn.decomposition import PCA
from sklearn.metrics import calinski_harabasz_score, davies_bouldin_score

# 各軸のデータを取り出して結合
x_data = all_data_scaled[:,:,0]
y_data = all_data_scaled[:,:,1]
z_data = all_data_scaled[:,:,2]

# データを結合してPCA入力用の2次元配列を作成
flattened_data = np.hstack((x_data, y_data, z_data))
print(f"Shape of flattened_data: {flattened_data.shape}")

# PCAによる次元削減
pca = PCA(n_components=2)
all_data_pca = pca.fit_transform(flattened_data)

# シルエットスコアの計算
silhouette_avg = silhouette_score(all_data_pca, labels, metric='euclidean')
print(f'平均シルエットスコア: {silhouette_avg}')

# 各サンプルのシルエットスコアの計算
silhouette_vals = silhouette_samples(all_data_pca, labels, metric='euclidean')

# シルエット係数のグラフのプロット
cluster_labels = np.unique(labels)
n_clusters = cluster_labels.shape[0]
y_ax_lower, y_ax_upper = 0, 0
yticks = []

for i, c in enumerate(cluster_labels):
    c_silhouette_vals = silhouette_vals[labels == c]
    c_silhouette_vals.sort()
    y_ax_upper += len(c_silhouette_vals)
    color = cm.jet(float(i) / n_clusters)
    plt.barh(range(y_ax_lower, y_ax_upper), c_silhouette_vals, height=1.0, edgecolor='none', color=color)
    yticks.append((y_ax_lower + y_ax_upper) / 2)
    y_ax_lower += len(c_silhouette_vals)
    print(f"クラスタ {c}: 要素数 {len(c_silhouette_vals)}, シルエット係数 {c_silhouette_vals}")

plt.axvline(silhouette_avg, color="red", linestyle="--")
plt.yticks(yticks, cluster_labels)
plt.ylabel('Cluster')
plt.xlabel('Silhouette coefficient')
plt.title('Silhouette Coefficients by Cluster (After PCA)')
plt.savefig("C:\\Users\\miyazakipc\\Desktop\\2024年9月10日 実験\\てすりとキネクト（てすり）\\データ分析\\Silhouette_coefficient.png")
plt.show()

# CH指標の計算
ch_score = calinski_harabasz_score(all_data_pca, labels)
print(f'Calinski-Harabasz 指標: {ch_score}')

# デービーズ・ボルダイン指数の計算
db_index = davies_bouldin_score(all_data_pca, labels)
print(f'Davies-Bouldin 指数: {db_index}')